In [1]:
%load_ext watermark
%watermark

Last updated: 2025-03-27T20:44:06.517811-04:00

Python implementation: CPython
Python version       : 3.12.9
IPython version      : 9.0.2

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 24.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit



In [2]:
import time

import jax
import jaxlib
import jaxopt
import numpy as np
import pandas as pd
import scipy
from multiprocess import pool

import pymedm
from pymedm import batch
from pymedm.diagnostics import moe_fit_rate
from pymedm.puma import puma

%watermark -w
%watermark -iv

Watermark: 2.5.0

pandas      : 2.2.3
pymedm      : 2.2.4.dev18+g1a367c4.d20250328
jaxopt      : 0.8.3
numpy       : 2.2.4
jaxlib      : 0.5.3
multiprocess: 0.70.17
jax         : 0.5.3
scipy       : 1.15.2



In [3]:
pymedm.processor_availability()

,device,available,count
0,cpu,yes,1
1,gpu,no,0
2,tpu,no,0


## Setup PUMAs

In [4]:
pumas = ["4701601", "4701602", "4701603", "4701604"]

In [5]:
mpu = dict.fromkeys(pumas)

In [6]:
for p in pumas:
    cind = pd.read_csv("../data/knox/cind_" + p + ".csv.gz", index_col=0)
    cg1 = pd.read_csv("../data/knox/cg1_" + p + ".csv.gz", index_col=0)
    cg2 = pd.read_csv("../data/knox/cg2_" + p + ".csv.gz", index_col=0)
    sg1 = pd.read_csv("../data/knox/sg1_" + p + ".csv.gz", index_col=0)
    sg2 = pd.read_csv("../data/knox/sg2_" + p + ".csv.gz", index_col=0)
    wt = pd.read_csv("../data/knox/wt_" + p + ".csv.gz", index_col=0).values.flatten()

    mpu[p] = puma(
        cind.index.values,
        2019,
        wt,
        cind,
        cg1,
        cg2,
        sg1,
        sg2,
    )

In [7]:
mpu

{'4701601': <pymedm.puma.puma at 0x16ea620c0>,
 '4701602': <pymedm.puma.puma at 0x17819ee10>,
 '4701603': <pymedm.puma.puma at 0x17b2faab0>,
 '4701604': <pymedm.puma.puma at 0x17b2af590>}

## Solve P-MEDM Problems

In [8]:
pmd_workers = pool.ThreadPool(60)
pmds = pmd_workers.map(
    lambda p: pymedm.PMEDM(
        mpu[p].est_ind.index,
        mpu[p].year,
        mpu[p].wt,
        mpu[p].est_ind,
        mpu[p].est_g1,
        mpu[p].est_g2,
        mpu[p].se_g1,
        mpu[p].se_g2,
        include_cg0=True,
        topo=mpu[p].topo,
        n_reps=0,
        random_state=int(p),
    ),
    pumas,
)
pmd_workers.close()

In [9]:
start_time = time.time()

pmds = batch.batch_solve(mpu)  # without allocation matrix replicates

exec_time = np.round(time.time() - start_time, 4)
print(f"\nCompleted in {exec_time} seconds.")


Completed in 35.4945 seconds.


In [10]:
pmds

{'4701601': <pymedm.pmedm.PMEDM at 0x56da9ddf0>,
 '4701602': <pymedm.pmedm.PMEDM at 0x31f31b0e0>,
 '4701603': <pymedm.pmedm.PMEDM at 0x56da9fe00>,
 '4701604': <pymedm.pmedm.PMEDM at 0x56da9fcb0>}

## Diagnostics

In [11]:
for p in mpu:
    print(f"PUMA {p}\n")

    mfrs = moe_fit_rate(mpu[p].est_ind, mpu[p].est_g2, mpu[p].se_g2, pmds[p].almat)

    print("MOE Fit Rate: " + str(np.round(mfrs["moe_fit_rate"], 4)))

    Ycomps = mfrs["Ycomp"]
    ycp = Ycomps[Ycomps.variable == "population"]
    ych = Ycomps[Ycomps.variable == "housing_units"]

    acs_totalpop = ycp.acs.sum()
    pmedm_totalpop = ycp.pmedm.sum()

    acs_totalhu = ych.acs.sum()
    pmedm_totalhu = ych.pmedm.sum()

    print(f"ACS Total Population: {acs_totalpop}")
    print(f"P-MEDM Total Population: {pmedm_totalpop}\n")
    print(f"ACS Total Housing Units: {acs_totalhu}")
    print(f"P-MEDM Total Units: {pmedm_totalhu}\n\n")

PUMA 4701601

MOE Fit Rate: 0.9997
ACS Total Population: 107250
P-MEDM Total Population: 107030.15776462702

ACS Total Housing Units: 49264
P-MEDM Total Units: 49289.171198960015


PUMA 4701602

MOE Fit Rate: 0.9996
ACS Total Population: 113236
P-MEDM Total Population: 112917.83541040667

ACS Total Housing Units: 47525
P-MEDM Total Units: 47639.18808323936


PUMA 4701603

MOE Fit Rate: 0.9992
ACS Total Population: 178350
P-MEDM Total Population: 177981.19198962985

ACS Total Housing Units: 87369
P-MEDM Total Units: 87773.65349391381


PUMA 4701604

MOE Fit Rate: 0.9998
ACS Total Population: 157817
P-MEDM Total Population: 157381.484036833

ACS Total Housing Units: 65814
P-MEDM Total Units: 65913.26547377858


